# Introduction to Linear Programming

#### Warmup

Using Gurobi, solve the following problem:  
    max $\; x+y+z$  
    s.t.  
    $x+y \leq 10$  
    $y + 3z \leq 2x$  
    $3x + 2y + z \leq 20$  
    $x, y, z \geq 0$  

In [1]:
import gurobipy as grb

env = grb.Env.ClientEnv("gurobi.log",
                        "10.116.90.112")
model = grb.Model("LP_Warmup", env)

Create Decision Variables

In [2]:
x = model.addVars(3, lb=0)

Set Objective 

In [3]:
model.setObjective(x[0] + x[1] + x[2], grb.GRB.MAXIMIZE)

Add Constraints

In [4]:
constr1 = model.addConstr(x[0] + x[1] <= 10)
constr2 = model.addConstr(x[1] + 3*x[2] <= 2*x[0])
constr3 = model.addConstr(3*x[0] + 2*x[1] + x[2] <= 20)

Optimize

In [5]:
model.optimize()
print(x)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 3 rows, 3 columns and 8 nonzeros
Model fingerprint: 0x29e5e92d
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 3 rows, 3 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+30   4.000000e+30   3.000000e+00      0s
       4    9.0909091e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.00 seconds
Optimal objective  9.090909091e+00
{0: <gurobi.Var C0 (value 5.454545454545455)>, 1: <gurobi.Var C1 (value 0.0)>, 2: <gurobi.Var C2 (value 3.6363636363636367)>}


In [6]:
del model
del env


Compute Server communication statistics:
  Sent: 0.003 MBytes in 20 msgs and 2.28s (0.00 MB/s)
  Received: 0.002 MBytes in 6 msgs and 0.63s (0.00 MB/s)



### Linear Programming

In linear programming, all functions (e.g. the objective and constraints) must be affine functions. This can be a considerable limitation, although in some cases a non-linear expression can be converted to a linear form (see Module 4). Although limiting, linear problems are generally much easier to solve than non-linear ones.

Linearity: $f(ax+by) = af(x) + bf(y)$

Affine: $f(x) = ax + b$

Note that an integrality constraint is not linear. If the other constraints are linear, this leads to integer linear programming (see Module 3).

Note, for decision variables $x$ and $y$, the following expressions are not linear:
 - $x \in \mathbb{Z}$
 - $xy$
 - $|x-y|$
 - $x (a-bx)$
 - $min(x,y)$

##### Simplex Algorithm

The simplex algorithm is an efficient algorithm, in practice, to solve linear programs.

Some definitions:
 - A linear inequality constraint is a linear hyperplane such that points lying on one side are feasible and on the other are not (a 'half-space').
 - The intersection of these half-spaces forms the feasible region.

For example, in two dimensions, the constraints $x \geq 0$,  $y \geq 0$,  $x \leq 1$,  $y \leq 1$ form a unit square. All feasible points lie within the square $[0,1]\times[0,1]$, which is a convex polytope.

If a unique optimal solution exists, then it must be found on a corner point (or extreme point) of the feasible space. If the optimal solution is not unique, then it may lie on an edge or corner point. In our square example, that means the solution $(x, y)$ must be one of the following if unique: $(0, 0)$, $(1, 0)$, $(0, 1)$, or $(1, 1)$.

The simplex algorithm efficiently traverses corner points of the feasible space until a better solution can no longer be found. This point is guaranteed to be optimal.

For more details, please see:
 - Wikipedia: [Simplex Algorithm](https://en.wikipedia.org/wiki/Simplex_algorithm)
 - Undergraduate Level: *Introduction to Operations Research* by Hillier and Lieberman 
 - Graduate Level: *Introduction to Linear Optimization* by Bertsimas and Tsitsiklis.

##### Other Considerations

- Gurobi uses a variety of algorithms, including the simplex. 
- The simplex algorithm always returns only one optimal solution. Other optimal solutions may exist.
- The optimal solution will always appear at a corner point of the feasible space for a linear program. This is generally not true for non-linear/integer problems. This is one of the reasons why linear programs are generally easier to solve to optimality.
- Adding variables or constraints to a linear program generally increases the complexity.



### RunDisney Example

Note: Fictional/Illustrative Example. All Numbers are Fabricated.

The Walt Disney World Marathon occurs every year in January. The Marathon Weekend is primarily composed of four races: a 5K on Thursday, a 10K on Friday, a half marathon on Saturday, and a marathon on Sunday. Participants can choose one of six options to participate as a runner: the four individual races plus the _Goofy's Race and a Half Challenge_ (the half marathon and marathon bundled) and the _Dopey Challenge_ (all four races bundled). 

For problems like this, it is useful to differentiate resources and products:
 - Resource - The raw unit that is being sold. In this example, the four race events are resources. 
 - Product - What the consumer is able to purchase. Products are comprised of one or more resources. Here the guests are able to purchase six different products that are comprised of the four individual events.

The following table contains the price and expected demand for each product.

|         Product |  Price | Demand |   
| --------------- | ------ | ------ |   
|              5K |  \\$80 |  15692 |   
|             10K | \\$120 |  11896 |     
|   Half Marathon | \\$180 |  18175 |    
|        Marathon | \\$190 |  17604 |    
| Goofy Challenge | \\$365 |   4325 |    
| Dopey Challenge | \\$560 |   5349 |  

Due to time and space constraints, only a limited number of runners are allowed for each race, and demand is often higher than capacity. As such, we would like to place limits on the number of available registrations for each product, while ensuring we make as much money as possible.

Each resource (race event) has a fixed capacity: the 5K and 10K are limited to 15,000 participants, while the half marathon and the marathon are limited to 25,000 participants.

The business has also requested that a specified number of each product be available. For example, because the 5K is open to children, they would like to reserve some spaces for 5K-only racers, as these spots can be quickly purchased by Dopey racers, who tend to book much earlier. These minimum values are shown in the table below.

|         Product |  Price | Demand | Min Spots |    
| --------------- | ------ | ------ | --------- |  
|              5K |  \\$80 |  15692 |     10000 |     
|             10K | \\$120 |  11896 |      7000 |  
|   Half Marathon | \\$180 |  18175 |      5000 |   
|        Marathon | \\$190 |  17604 |      5000 |     
| Goofy Challenge | \\$365 |   4325 |      2000 |  
| Dopey Challenge | \\$560 |   5349 |      2000 |  

The data below can be used as input for the optimization model:

In [7]:
import gurobipy as grb
import pandas as pd

# Input Data 
# Product Information
prod_df = pd.DataFrame({
    'Product': ['5K', '10K', 'Half', 'Marathon', 'Goofy', 'Dopey'],
    'Price': [80, 120, 180, 190, 365, 560],
    'Demand': [15692, 11896, 18175, 17604, 4325, 5349],
    'Min_Spots': [10000, 7000, 5000, 5000, 2000, 2000],
})

prod_df.set_index('Product', inplace=True)
prod_list = prod_df.index.values

# Resource Information
res_df = pd.DataFrame({
    'Resource': ['5K', '10K', 'Half', 'Marathon'],
    'Capacity': [15000, 15000, 25000, 25000],
    'Products': [['5K', 'Dopey'], ['10K', 'Dopey'], 
                 ['Half', 'Goofy', 'Dopey'], ['Marathon', 'Goofy', 'Dopey']],
})

res_df.set_index('Resource', inplace=True)
res_list = res_df.index.values

print(prod_df)
print(res_df)

          Price  Demand  Min_Spots
Product                           
5K           80   15692      10000
10K         120   11896       7000
Half        180   18175       5000
Marathon    190   17604       5000
Goofy       365    4325       2000
Dopey       560    5349       2000
          Capacity                  Products
Resource                                    
5K           15000               [5K, Dopey]
10K          15000              [10K, Dopey]
Half         25000      [Half, Goofy, Dopey]
Marathon     25000  [Marathon, Goofy, Dopey]


In [14]:
env = grb.Env.ClientEnv("gurobi.log",
                        "10.116.90.112")
model = grb.Model("Marathon", env)


Compute Server communication statistics:
  Sent: 0.001 MBytes in 8 msgs and 0.85s (0.00 MB/s)
  Received: 0.001 MBytes in 4 msgs and 0.40s (0.00 MB/s)



###### What are the decision variables?

The number of spots to open up for registration for each product. 

$x_i$ = number of spots for product $i \in$ {5K, 10K, Half Marathon, Marathon, Goofy, Dopey}

In [15]:
spots = model.addVars(prod_list, lb=0)

###### What is the objective?

Maxmize revenue.

$max_{x_i} \sum_{i} p_i*x_i$

where $p_i$ = price for product $i$.

In [16]:
exp_rev = grb.quicksum( 
    spots[prod]*prod_df['Price'][prod] for prod in prod_list )

total_exp_rev = model.setObjective(exp_rev, grb.GRB.MAXIMIZE)

###### What are the constraints?

Constraint: Open up spots equal to or greater than the requested minimum.

$x_i \geq m_i \; \forall i$

where $m_i$ = minimum number of spots for product $i$.

In [17]:
min_spot_constraint = model.addConstrs((spots[prod] >= prod_df['Min_Spots'][prod] for prod in prod_list), 
                                       'min_spot_constraint')

The following constraints may seem slightly silly. Our model will assume that demand *exactly equals* estimated demand, which is obviously wrong. However, in practice, it can be quite difficult to accurately model uncertainty in an optimization model, and this assumption is often made in practice. Methods for handling uncertainty, such as robust or stochastic optimization, are outside the scope of this group.

Constraint: Allow no more spots than capacity will allow.

$\sum_{i \in S_j} x_i \leq c_j \; \forall j$

where $S_j$ are the sets of products that consume resource (race) $j$ and $c_j$ is the capacity for race $j$

e.g. $S_{5K}$ = {5K, Dopey}

In [18]:
capacity_constraint = model.addConstrs((grb.quicksum(spots[prod] for prod in res_df['Products'][res]) 
                                        <= res_df['Capacity'][res] for res in res_list), 
                                       'capacity_constraint')

Constraint: Do no open up more spots than demand.

According to our previous demand assumption, opening up any more spots is futile, as there will be no further demand. Thus it makes sense to cap number of spots so guests can purchase other products when demand is met.

$x_i \leq d_i \; \forall i$

where $d_i$ is the demand for product $i$. 

In [19]:
demand_constraint = model.addConstrs((spots[prod] <= prod_df['Demand'][prod] for prod in prod_list), 
                                     'demand_constraint')

Optimize and generate output.

In [20]:
model.optimize()

# Generate output as a data frame
spots_values = model.getAttr('x',spots)
prod_df['opt_spots'] = pd.Series(spots_values)

print(prod_df)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 16 rows, 6 columns and 22 nonzeros
Model fingerprint: 0x14bd901b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+01, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 2e+04]
Presolve removed 13 rows and 1 columns
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4074885e+07   1.678625e+03   0.000000e+00      0s
       2    1.2197500e+07   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds
Optimal objective  1.219750000e+07
          Price  Demand  Min_Spots  opt_spots
Product                                      
5K           80   15692      10000    11896.0
10K         120   11896       7000    11896.0
Half        180   18175       5000    17604.0
Marathon    190   17604       5000    17604.0
Goofy       365    4325       2000     4292.0
Dopey 

In [21]:
del model
del env


Compute Server communication statistics:
  Sent: 0.003 MBytes in 18 msgs and 1.79s (0.00 MB/s)
  Received: 0.002 MBytes in 6 msgs and 0.53s (0.00 MB/s)

